In [20]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install streamlit

Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [23]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install sqlalchemy

In [26]:
import requests
import pandas as pd
import psycopg2

In [27]:
#url
url = "https://api.sportradar.com/tennis/trial/v3/en/competitions.json?api_key=KO9UV9wO215z329teE9CYAfqfZXjq8Pbiqzzvw1X"

# API request
def fetch_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Error-statuscode: {response.status_code}")
try:
    data = fetch_data(url)#json--->pydic

    competitions = data.get("competitions", [])

    competition_details = []#competition
    category_details = []#category

    for competition in competitions:
        comp_detail = {
            "competition_id": competition.get("id"),
            "competition_name": competition.get("name"),
            "parent_id":competition.get("parent_id"),
            "type": competition.get("type"),
            "gender": competition.get("gender"),
            "category_id" : competition.get("category",{}).get("id"),
        }
        competition_details.append(comp_detail)

        
        category = competition.get("category", {})
        category_detail = {
            "category_id": category.get("id"),
            "category_name": category.get("name"),
        }
        category_details.append(category_detail)
    print("data inserted succesfully")
except Exception as e:
    print(failed)

data inserted succesfully


In [28]:
#url
url = "https://api.sportradar.com/tennis/trial/v3/en/complexes.json?api_key=KO9UV9wO215z329teE9CYAfqfZXjq8Pbiqzzvw1X"

# API request
def fetch_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Error-statuscode: {response.status_code}")
try:
    data=fetch_data(url)#json--->pydic

    complex_details = []#complex
    venue_details = []#venue

    complexes = data.get("complexes", [])
    for complex_item in complexes:
        comp_details={
            "complex_id":complex_item.get("id"),
            "complex_name":complex_item.get("name")
        }
        complex_details.append(comp_details)

        venues = complex_item.get("venues", [])
        for venue in venues:
                venue_detail = {
                    "venue_id": venue.get("id"),
                    "venue_name": venue.get("name"),
                    "city_name": venue.get("city_name"),
                    "country_name": venue.get("country_name"),
                    "country_code": venue.get("country_code"),
                    "timezone": venue.get("timezone"),
                    "complex_id": complex_item.get("id")  
                }
                venue_details.append(venue_detail)
    print("data inserted succesfully")
except Exception as e:
    print(failed)

data inserted succesfully


In [81]:
import requests
import mysql.connector

# URL
url = "https://api.sportradar.com/tennis/trial/v3/en/double_competitors_rankings.json?api_key=KO9UV9wO215z329teE9CYAfqfZXjq8Pbiqzzvw1X"

# API request
def fetch_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Error: {response.status_code}")

try:
    data = fetch_data(url)  # JSON -> Python dict

    Competitors_Table = []  # competitor
    Competitor_Rankings_Table = []  # competitor_rank

    for ranking in data.get("rankings", []):
        for competitor_ranking in ranking.get("competitor_rankings", []):
            Rank_data = {
                "rank": competitor_ranking.get("rank"),
                "movement": competitor_ranking.get("movement"),
                "points": competitor_ranking.get("points"),
                "competitions_played": competitor_ranking.get("competitions_played"),
                "competitor_id": competitor_ranking["competitor"].get("id"),
            }
            Competitor_Rankings_Table.append(Rank_data)

            compi = competitor_ranking.get("competitor", {})
            compx_info = {
                "competitor_id": compi.get("id"),
                "name": compi.get("name"),
                "country": compi.get("country"),
                "country_code": compi.get("country_code"),
                "abbreviation": compi.get("abbreviation")
            }
            Competitors_Table.append(compx_info)

    # Insert data into MySQL
    for competitor in Competitors_Table:
        try:
            mycursor.execute("""
                INSERT IGNORE INTO competitors (competitor_id, name, country, country_code, abbreviation)
                VALUES (%s, %s, %s, %s, %s)
            """, (
                competitor['competitor_id'],
                competitor['name'],
                competitor['country'],
                competitor['country_code'],
                competitor['abbreviation']
            ))
            mydb.commit()
        except Exception as e:
            print(f"Competitor Insert Error: {e}")

    for ranking in Competitor_Rankings_Table:
        try:
            mycursor.execute("""
                INSERT INTO competitor_rankings (rank, movement, points, competitions_played, competitor_id)
                VALUES (%s, %s, %s, %s, %s)
            """, (
                ranking['rank'],
                ranking['movement'],
                ranking['points'],
                ranking['competitions_played'],
                ranking['competitor_id']
            ))
            mydb.commit()
        except Exception as e:
            print(f"Ranking Insert Error: {e}")

    print("Data inserted successfully")

except Exception as e:
    print(e)

Data inserted successfully


In [48]:
pd.DataFrame(competition_details).to_csv('competition_details.csv', index=False)
pd.DataFrame(category_details).to_csv('category_details.csv',index=False)

In [49]:
pd.DataFrame(complex_details).to_csv("complex_details.csv",index=False)
pd.DataFrame(venue_details).to_csv("venue_details.csv",index=False)

In [50]:
pd.DataFrame(Competitor_Rankings_Table).to_csv('Competitor_Rankings_Table.csv',index=False)
pd.DataFrame(Competitors_Table).to_csv('Competitors_Table.csv',index=False)

In [51]:
import mysql.connector

In [52]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
)
print(mydb)
mycursor = mydb.cursor(buffered=True)

In [53]:
mycursor.execute("SHOW DATABASES")

for x in mycursor:
    print(x)

('dj_group',)
('information_schema',)
('mdtm40',)
('mysql',)
('performance_schema',)
('phpmyadmin',)
('tennis_db',)
('test',)


In [36]:
mycursor.execute("CREATE DATABASE tennis_db")

In [54]:
mycursor.execute("use tennis_db")

In [55]:
mycursor.execute("""
    CREATE TABLE categories (
        category_id VARCHAR(50) PRIMARY KEY,
        category_name VARCHAR(100));""")
print("table_1 categories created columns added")

mycursor.execute("""
    create table competitions(
    competition_id varchar(50) primary key,
    competition_name varchar(100) ,
    parent_id varchar(50) ,
    type varchar(20),
    gender varchar(10) ,
    category_id varchar(50),
    foreign key (category_id) references categories(category_id));""")
print("table_2 competition created columns added")

table_1 categories created columns added
table_2 competition created columns added


In [56]:
mycursor.execute("""
CREATE TABLE complexes (
    complex_id VARCHAR(50)  PRIMARY KEY,
    complex_name VARCHAR(100));""")
print("table_3 complexes created columns added")

mycursor.execute("""
CREATE TABLE venue (
    venue_id varchar(50) PRIMARY KEY, 
    venue_name varchar(100),
    city_name varchar(100) ,
    country_name varchar(100) ,
    country_code char(3),
    timezone varchar(100),
    complex_id varchar(50),
    FOREIGN KEY (complex_id) REFERENCES complexes(complex_id));""")
print("Table_4 venues created columns added")

table_3 complexes created columns added
Table_4 venues created columns added


In [57]:
mycursor.execute("""
CREATE TABLE Competitors (
    competitor_id VARCHAR(50) PRIMARY KEY,
    name VARCHAR(100),
    country VARCHAR(100),
    country_code CHAR(3),
    abbreviation VARCHAR(10));""")
print("table_5 competitors created column added")

mycursor.execute("""
CREATE TABLE Competitor_Rankings (
    rank_id SERIAL PRIMARY KEY, 
    rank int,
    movement INT ,
    points INT ,
    competitions_played INT ,
    competitor_id VARCHAR(50),
    FOREIGN KEY (competitor_id) REFERENCES Competitors(competitor_id));""")
print("table_6 competitor_ranking created column added")

table_5 competitors created column added
table_6 competitor_ranking created column added


In [58]:
df = pd.read_csv('category_details.csv')
df1=pd.read_csv('competition_details.csv')
df2=pd.read_csv("complex_details.csv")
df3=pd.read_csv("venue_details.csv")
df4=pd.read_csv('Competitors_Table.csv')
df5=pd.read_csv('Competitor_Rankings_Table.csv')
df5['movement'] = df5['movement'].fillna(0)

In [59]:
insert = """
INSERT IGNORE INTO categories (category_id, category_name)
VALUES (%s, %s)
"""

for index, row in df.iterrows():
    mycursor.execute(insert, (row['category_id'], row['category_name']))

mydb.commit()

insert_query = """
INSERT INTO competitions (competition_id, competition_name, parent_id, type, gender, category_id)
VALUES (%s, %s, %s, %s, %s, %s)
ON DUPLICATE KEY UPDATE
    competition_name = VALUES(competition_name),
    parent_id = VALUES(parent_id),
    type = VALUES(type),
    gender = VALUES(gender),
    category_id = VALUES(category_id)
"""

df1 = df1.where(pd.notnull(df1), None)

for index, row in df1.iterrows():
    try:
        mycursor.execute(insert_query, (
            row['competition_id'], 
            row['competition_name'], 
            row['parent_id'], 
            row['type'], 
            row['gender'], 
            row['category_id']
        ))
    except Exception as e:
        print(f"Error at row {index}: {e}")

mydb.commit()

print("Data inserted successfully.")

Data inserted successfully.


In [60]:
insert_complex = """
INSERT IGNORE INTO complexes (complex_id, complex_name)
VALUES (%s, %s)
"""

for index, row in df2.iterrows():
        mycursor.execute(insert_complex, (row['complex_id'], row['complex_name']))

mydb.commit()

insert_venue = """
INSERT INTO venue (venue_id, venue_name, city_name, country_name, country_code, timezone, complex_id)
VALUES (%s, %s, %s, %s, %s, %s, %s)
ON DUPLICATE KEY UPDATE
    venue_name = VALUES(venue_name),
    city_name = VALUES(city_name),
    country_name = VALUES(country_name),
    country_code = VALUES(country_code),
    timezone = VALUES(timezone),
    complex_id = VALUES(complex_id)
"""

df3 = df3.where(pd.notnull(df3), None)

for index, row in df3.iterrows():
    try:
        mycursor.execute(insert_venue, (
            row['venue_id'], 
            row['venue_name'], 
            row['city_name'], 
            row['country_name'], 
            row['country_code'], 
            row['timezone'], 
            row['complex_id']
        ))
    except Exception as e:
        print(f"Error at row {index} (venue): {e}")

mydb.commit()

print("data successfully inserted")

data successfully inserted


In [61]:
df4 = df4.where(pd.notnull(df4), None)

insert = """ 
INSERT IGNORE INTO competitors (competitor_id, name, country, country_code, abbreviation)
VALUES (%s, %s, %s, %s, %s)
"""

for index, row in df4.iterrows():
    try:
        mycursor.execute(insert, (
            row['competitor_id'], 
            row['name'], 
            row['country'], 
            row['country_code'], 
            row['abbreviation']
        ))
    except Exception as e:
        print(f"Error at row {index}: {e}")

mydb.commit()
print("data inserted successfully.")

data inserted successfully.


In [62]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.


In [63]:
import tabulate

In [64]:
#Table 1 ---> Data Analysis

In [82]:
from tabulate import tabulate

q1 = """
SELECT 
    c.competition_id AS 'Competition ID',
    c.competition_name AS 'Competition Name',
    c.type AS 'Type',
    c.gender AS 'Gender',
    c.category_id AS 'Category ID',
    cat.category_name AS 'Category Name'
FROM 
    competitions c 
JOIN 
    categories cat ON c.category_id = cat.category_id;
"""

mycursor.execute(q1)
rows = mycursor.fetchall()
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(rows, columns=columns)

print("1. Competitions along with their Category Name:\n")
print(tabulate(df, headers='keys', tablefmt='psql', showindex=False))

1. Competitions along with their Category Name:

+----------------------+--------------------------------------------------------------------+---------------+----------+------------------+----------------------+
| Competition ID       | Competition Name                                                   | Type          | Gender   | Category ID      | Category Name        |
|----------------------+--------------------------------------------------------------------+---------------+----------+------------------+----------------------|
| sr:competition:12229 | IPTL                                                               | singles       | men      | sr:category:1012 | IPTL                 |
| sr:competition:14814 | Juniors AO, Melbourne, Australia Men Singles                       | singles       | men      | sr:category:1474 | Juniors              |
| sr:competition:14816 | Juniors AO, Melbourne, Australia Men Doubles                       | doubles       | men      | sr:category:147

In [83]:
from tabulate import tabulate

q2 = """
SELECT 
    cat.category_name AS 'Category Name', 
    COUNT(c.competition_id) AS 'Number of Competitions'
FROM 
    competitions c 
JOIN  
    categories cat ON c.category_id = cat.category_id
GROUP BY 
    cat.category_name;
"""

mycursor.execute(q2)
out_2 = mycursor.fetchall()

print("\n2. Number of competitions in each category:\n")
print(tabulate(out_2, headers=["Category Name", "Number of Competitions"], tablefmt="psql"))


2. Number of competitions in each category:

+----------------------+--------------------------+
| Category Name        |   Number of Competitions |
|----------------------+--------------------------|
| ATP                  |                      223 |
| Billie Jean King Cup |                        1 |
| Challenger           |                      900 |
| Davis Cup            |                        1 |
| Exhibition           |                       32 |
| Hopman Cup           |                        1 |
| IPTL                 |                        1 |
| ITF Men              |                     2198 |
| ITF Women            |                     2032 |
| Juniors              |                       16 |
| Legends              |                       11 |
| United Cup           |                        1 |
| UTR Men              |                       80 |
| UTR Women            |                       72 |
| Wheelchairs          |                       16 |
| Wheelchairs Juni

In [84]:
from tabulate import tabulate

query3 = """
SELECT 
    competition_id,
    competition_name,
    type
FROM 
    competitions
WHERE 
    type = 'doubles';
"""
mycursor.execute(query3)
result3 = mycursor.fetchall()

# Convert the result to a table format
headers = ["Competition ID", "Competition Name", "Type"]
table = tabulate(result3, headers=headers, tablefmt="psql")

print("\n3. All competitions of type 'doubles':")
print(table)


3. All competitions of type 'doubles':
+----------------------+--------------------------------------------------------------------+---------+
| Competition ID       | Competition Name                                                   | Type    |
|----------------------+--------------------------------------------------------------------+---------|
| sr:competition:10027 | ITF Men Stara Zagora, Bulgaria Men Doubles                         | doubles |
| sr:competition:10033 | ITF Men Sibiu, Romania Men Doubles                                 | doubles |
| sr:competition:10039 | ITF Men Busto Arsizio, Italy Men Doubles                           | doubles |
| sr:competition:10045 | ITF Men Sabac, Serbia Men Doubles                                  | doubles |
| sr:competition:10057 | ITF Men Seefeld, Austria Men Doubles                               | doubles |
| sr:competition:10075 | ATP Challenger Portoroz, Slovenia Men DoublesUTR Wo                | doubles |
| sr:competition:10093 |

In [85]:
from tabulate import tabulate

category_id = 'sr:category:3'  
query4 = f"""
SELECT 
    competition_id,
    competition_name,
    category_id
FROM 
    competitions
WHERE 
    category_id = '{category_id}';
"""
mycursor.execute(query4)
result4 = mycursor.fetchall()

# Convert the result to a table format
headers = ["Competition ID", "Competition Name", "Category ID"]
table = tabulate(result4, headers=headers, tablefmt="psql")

print(f"\n4. Competitions of category {category_id}:")
print(table)


4. Competitions of category sr:category:3:
+----------------------+----------------------------------------------+---------------+
| Competition ID       | Competition Name                             | Category ID   |
|----------------------+----------------------------------------------+---------------|
| sr:competition:11123 | ATP Shenzhen, China Men Singles              | sr:category:3 |
| sr:competition:11125 | ATP Shenzhen, China Men Doubles              | sr:category:3 |
| sr:competition:11129 | ATP Rio de Janeiro, Brazil Men Singles       | sr:category:3 |
| sr:competition:11131 | ATP Rio de Janeiro, Brazil Men Doubles       | sr:category:3 |
| sr:competition:12315 | ATP Geneva, Switzerland Men Singles          | sr:category:3 |
| sr:competition:12317 | ATP Geneva, Switzerland Men Doubles          | sr:category:3 |
| sr:competition:12321 | ATP Istanbul, Turkey Men Singles             | sr:category:3 |
| sr:competition:12323 | ATP Istanbul, Turkey Men Doubles             | sr:c

In [86]:
from tabulate import tabulate

query5 = """
SELECT 
    parent.competition_id AS parent_id,
    parent.competition_name AS parent_name,
    sub.competition_id AS sub_competition_id,
    sub.competition_name AS sub_competition_name
FROM 
    competitions parent
JOIN 
    competitions sub ON parent.competition_id = sub.parent_id
WHERE 
    parent.parent_id IS NULL;
"""
mycursor.execute(query5)
result5 = mycursor.fetchall()

# Convert the result to a table format
headers = ["Parent ID", "Parent Name", "Sub-Competition ID", "Sub-Competition Name"]
table = tabulate(result5, headers=headers, tablefmt="psql")

print("\n5. Parent competitions and their sub-competitions:")
print(table)


5. Parent competitions and their sub-competitions:
+----------------------+--------------------+----------------------+------------------------+
| Parent ID            | Parent Name        | Sub-Competition ID   | Sub-Competition Name   |
|----------------------+--------------------+----------------------+------------------------|
| sr:competition:45363 | UTR Boca Raton W01 | sr:competition:45971 | UTR Boca Raton W03     |
+----------------------+--------------------+----------------------+------------------------+


In [87]:
from tabulate import tabulate

query6 = """
SELECT 
    cat.category_name,
    c.type,
    COUNT(c.competition_id) AS competition_count
FROM 
    competitions c
JOIN 
    categories cat ON c.category_id = cat.category_id
GROUP BY 
    cat.category_name, c.type
ORDER BY 
    cat.category_name, c.type;
"""
mycursor.execute(query6)
result6 = mycursor.fetchall()

# Convert the result to a table format
headers = ["Category Name", "Competition Type", "Count"]
table = tabulate(result6, headers=headers, tablefmt="psql", stralign="left", numalign="center")

print("\n6. Distribution of competition types by category:")
print(table)


6. Distribution of competition types by category:
+----------------------+--------------------+---------+
| Category Name        | Competition Type   |  Count  |
|----------------------+--------------------+---------|
| ATP                  | doubles            |   107   |
| ATP                  | mixed              |    2    |
| ATP                  | mixed_doubles      |    6    |
| ATP                  | singles            |   108   |
| Billie Jean King Cup | mixed              |    1    |
| Challenger           | doubles            |   449   |
| Challenger           | singles            |   451   |
| Davis Cup            | mixed              |    1    |
| Exhibition           | doubles            |    8    |
| Exhibition           | mixed_doubles      |    2    |
| Exhibition           | singles            |   22    |
| Hopman Cup           | mixed              |    1    |
| IPTL                 | singles            |    1    |
| ITF Men              | doubles            |  1099  

In [88]:
from tabulate import tabulate

query7 = """
SELECT 
    competition_id,
    competition_name
FROM 
    competitions
WHERE 
    parent_id IS NULL;
"""
mycursor.execute(query7)
result7 = mycursor.fetchall()

headers = ["Competition ID", "Competition Name"]
table = tabulate(
    result7, 
    headers=headers, 
    tablefmt="psql",
)

print("\n7. All competitions with no parent (top-level competitions):")
print(table)


7. All competitions with no parent (top-level competitions):
+----------------------+---------------------------------------------+
| Competition ID       | Competition Name                            |
|----------------------+---------------------------------------------|
| sr:competition:1207  | Championship International Series           |
| sr:competition:12229 | IPTL                                        |
| sr:competition:15393 | ITF Men San Jose, Costa Rica Men Singles    |
| sr:competition:15395 | ITF Men San Jose, Costa Rica Men Doubles    |
| sr:competition:2100  | Davis Cup                                   |
| sr:competition:2102  | Billie Jean King Cup                        |
| sr:competition:21028 | ITF Colombia 02A, Women Singles             |
| sr:competition:21228 | ITF Australia 02B, Women Doubles            |
| sr:competition:22334 | ITF Uzbekistan 01A, Women Singles           |
| sr:competition:22763 | ITF Tunisia 39A, Women Doubles              |
| sr:competitio

In [89]:
#Table 2 ---> Data Analysis

In [90]:
from tabulate import tabulate

query1 = """
SELECT 
    v.venue_name, 
    c.complex_name
FROM 
    venue v
LEFT JOIN 
    complexes c 
ON 
    v.complex_id = c.complex_id;
"""
mycursor.execute(query1)
result1 = mycursor.fetchall()

# Format as table
headers = ["Venue Name", "Associated Complex"]
table = tabulate(
    result1, 
    headers=headers,
    tablefmt="psql",
)

print("\n 1.Venues along with their associated complex name:")
print(table)


 1.Venues along with their associated complex name:
+------------------------------------------------------------+------------------------------------------------------------+
| Venue Name                                                 | Associated Complex                                         |
|------------------------------------------------------------+------------------------------------------------------------|
| Estadio Monumental                                         | Club Tennis Las Terrazas de Miraflores                     |
| Stadium                                                    | Shenzhen International Tennis Center                       |
| Court 2                                                    | Shenzhen Longgang Sports Center                            |
| Court 7                                                    | Olympic Park Tennis Center                                 |
| Court 20                                                   | Melbourne Park  

In [91]:
from tabulate import tabulate

query2 = """
SELECT 
    c.complex_name, 
    COUNT(v.venue_id) AS venue_count
FROM 
    complexes c
LEFT JOIN 
    venue v 
ON 
    c.complex_id = v.complex_id
GROUP BY 
    c.complex_name
ORDER BY 
    venue_count DESC;
"""
mycursor.execute(query2)
result2 = mycursor.fetchall()

headers = ["Complex Name", "Number of Venues"]
table = tabulate(
    result2,
    headers=headers,
    tablefmt="psql",
)
print("\n 2.Number of venues in each complex:")
print(table)


 2.Number of venues in each complex:
+------------------------------------------------------------+--------------------+
| Complex Name                                               |   Number of Venues |
|------------------------------------------------------------+--------------------|
| National Tennis Center                                     |                 56 |
| Buenos Aires Lawn Tennis Club                              |                 30 |
| Melbourne Park                                             |                 25 |
| Club Tennis Las Terrazas de Miraflores                     |                 22 |
| Qi Zhong Tennis Center                                     |                 21 |
| Indian Wells Tennis Garden                                 |                 21 |
| Megasaray Tennis Academy                                   |                 21 |
| Ariake Tennis Forest Park Ariake Colesseum                 |                 21 |
| Clube Hipico Santo Amaro            

In [92]:
from tabulate import tabulate

query3 = """
SELECT 
    venue_name, 
    city_name, 
    country_name, 
    timezone
FROM 
    venue
WHERE 
    country_name = 'australia'
ORDER BY
    city_name, venue_name;
"""
mycursor.execute(query3)
result3 = mycursor.fetchall()

headers = ["Venue", "City", "Country", "Timezone"]
table = tabulate(
    result3,
    headers=headers,
    tablefmt="psql",
)

print("\n 3.Venue details in australia:")
print(table)


 3.Venue details in australia:
+-----------------------------------------+---------------+-----------+---------------------+
| Venue                                   | City          | Country   | Timezone            |
|-----------------------------------------+---------------+-----------+---------------------|
| Centre Court                            | Adelaide      | Australia | Australia/South     |
| Court 2                                 | Adelaide      | Australia | Australia/South     |
| Court 3                                 | Adelaide      | Australia | Australia/South     |
| Court 4                                 | Adelaide      | Australia | Australia/South     |
| Show Court 1                            | Adelaide      | Australia | Australia/South     |
| Court 1                                 | Brisbane      | Australia | Australia/Brisbane  |
| Court 10                                | Brisbane      | Australia | Australia/Brisbane  |
| Court 14                  

In [93]:
from tabulate import tabulate

query4 = """
SELECT 
    venue_name, 
    timezone
FROM 
    venue
ORDER BY
    venue_name ASC;
"""
mycursor.execute(query4)
result4 = mycursor.fetchall()

headers = ["Venue Name", "Timezone"]
table = tabulate(
    result4,
    headers=headers,
    tablefmt="psql",
)

print("\n 4.Venues and Their Timezones:")
print(table)


 4.Venues and Their Timezones:
+------------------------------------------------------------+--------------------------------+
| Venue Name                                                 | Timezone                       |
|------------------------------------------------------------+--------------------------------|
| 12BET COURT                                                | Europe/London                  |
| 1573 Arena                                                 | Australia/Melbourne            |
| Abierto Akron Zapopan                                      | America/Mexico_City            |
| Acqua Maniva                                               | Europe/Rome                    |
| ADCB Court 1                                               | Asia/Dubai                     |
| AJC Grandstand Court                                       | America/New_York               |
| Akron Tennis Stadium                                       | America/Mexico_City            |
| Alaadd

In [94]:
from tabulate import tabulate

query5 = """
SELECT 
    c.complex_name, 
    COUNT(v.venue_id) AS venue_count
FROM 
    complexes c
JOIN 
    venue v 
ON 
    c.complex_id = v.complex_id
GROUP BY 
    c.complex_name
HAVING 
    COUNT(v.venue_id) > 1
ORDER BY 
    venue_count DESC;
"""
mycursor.execute(query5)
result5 = mycursor.fetchall()

headers = ["Complex Name", "Number of Venues"]
table = tabulate(
    result5,
    headers=headers,
    tablefmt="psql",
)

print("\n 5.Complexes with Multiple Venues:")
print(table)


 5.Complexes with Multiple Venues:
+------------------------------------------------------------+--------------------+
| Complex Name                                               |   Number of Venues |
|------------------------------------------------------------+--------------------|
| National Tennis Center                                     |                 56 |
| Buenos Aires Lawn Tennis Club                              |                 30 |
| Melbourne Park                                             |                 25 |
| Club Tennis Las Terrazas de Miraflores                     |                 22 |
| Megasaray Tennis Academy                                   |                 21 |
| Ariake Tennis Forest Park Ariake Colesseum                 |                 21 |
| Qi Zhong Tennis Center                                     |                 21 |
| Indian Wells Tennis Garden                                 |                 21 |
| Queensland Tennis Centre              

In [95]:
query6 = """
SELECT 
    country_name, 
    ARRAY_AGG(venue_name) AS venues
FROM 
    venue
GROUP BY 
    country name
"""

mycursor.execute(query5)
result5 = mycursor.fetchall()

headers = ["venue name", "country name"]
table = tabulate(
    result5,
    headers=headers,
    tablefmt="psql",
)
print("\n 6.Venues Grouped by Country:")
print(table)


 6.Venues Grouped by Country:
+------------------------------------------------------------+----------------+
| venue name                                                 |   country name |
|------------------------------------------------------------+----------------|
| National Tennis Center                                     |             56 |
| Buenos Aires Lawn Tennis Club                              |             30 |
| Melbourne Park                                             |             25 |
| Club Tennis Las Terrazas de Miraflores                     |             22 |
| Megasaray Tennis Academy                                   |             21 |
| Ariake Tennis Forest Park Ariake Colesseum                 |             21 |
| Qi Zhong Tennis Center                                     |             21 |
| Indian Wells Tennis Garden                                 |             21 |
| Queensland Tennis Centre                                   |             20 |
| Clube H

In [96]:
from tabulate import tabulate
complex_name = 'Palais des sports de Gerland'

query7 = f"""
SELECT 
    venue_name
FROM 
    venue
WHERE 
    complex_id IN (SELECT complex_id FROM complexes WHERE complex_name = '{complex_name}');
"""
mycursor.execute(query7)
result7 = mycursor.fetchall()
headers = [f"Venues at {complex_name}"]
table = tabulate(
    result7,
    headers=headers,
    tablefmt="psql",  # Clean rounded-corner style
    stralign="left",   # Left-align venue names
    missingval="None"  # For any NULL values
)

print(f"\n 7.Venues at {complex_name}:")
print(table)


 7.Venues at Palais des sports de Gerland:
+------------------------------------------+
| Venues at Palais des sports de Gerland   |
|------------------------------------------|
| Indoor 1                                 |
| Indoor 2                                 |
| Indoor 3                                 |
| Court 23                                 |
| Court 14                                 |
| Court 15                                 |
| Court 12                                 |
+------------------------------------------+


In [97]:
#Table3 ---> Data Analysis

In [98]:
# Database connection
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="tennis_db"
)
mycursor = mydb.cursor(buffered=True)

# Function to execute and display queries in psql format
def execute_and_display(query, description):
    print(f"\n{description}")
    print("-" * 50)
    mycursor.execute(query)
    result = mycursor.fetchall()
    
    # Get column names
    column_names = [i[0] for i in mycursor.description]
    
    # Display in psql table format
    print(tabulate(result, headers=column_names, tablefmt="psql"))

query1 = """
SELECT c.name, cr.rank, cr.points, c.country
FROM Competitor_Rankings cr
JOIN Competitors c ON cr.competitor_id = c.competitor_id
ORDER BY cr.rank
"""

execute_and_display(query1, "1. All competitors with their rank and points")


1. All competitors with their rank and points
--------------------------------------------------
+--------------------------------------+--------+----------+--------------------------+
| name                                 |   rank |   points | country                  |
|--------------------------------------+--------+----------+--------------------------|
| Pavic, Mate                          |      1 |     9350 | Croatia                  |
| Siniakova, Katerina                  |      1 |    10665 | Czechia                  |
| Arevalo-Gonzalez, Marcelo            |      1 |     9350 | El Salvador              |
| Townsend, Taylor                     |      2 |     8825 | USA                      |
| Heliovaara, Harri                    |      3 |     7590 | Finland                  |
| Routliffe, Erin                      |      3 |     8125 | New Zealand              |
| Ostapenko, Jelena                    |      4 |     6775 | Latvia                   |
| Patten, Henry       

In [99]:

query2 = """
SELECT c.name, cr.rank, cr.points, c.country
FROM Competitor_Rankings cr
JOIN Competitors c ON cr.competitor_id = c.competitor_id
WHERE cr.rank <= 5
ORDER BY cr.rank
"""

execute_and_display(query2, "2. Competitors ranked in the top 5")


2. Competitors ranked in the top 5
--------------------------------------------------
+---------------------------+--------+----------+---------------+
| name                      |   rank |   points | country       |
|---------------------------+--------+----------+---------------|
| Pavic, Mate               |      1 |     9350 | Croatia       |
| Siniakova, Katerina       |      1 |    10665 | Czechia       |
| Arevalo-Gonzalez, Marcelo |      1 |     9350 | El Salvador   |
| Townsend, Taylor          |      2 |     8825 | USA           |
| Heliovaara, Harri         |      3 |     7590 | Finland       |
| Routliffe, Erin           |      3 |     8125 | New Zealand   |
| Patten, Henry             |      4 |     7590 | Great Britain |
| Ostapenko, Jelena         |      4 |     6775 | Latvia        |
| Thompson, Jordan          |      5 |     6460 | Australia     |
| Dabrowski, Gabriela       |      5 |     6385 | Canada        |
+---------------------------+--------+----------+------

In [100]:
query3 = """
SELECT c.name, cr.rank, cr.movement, cr.points, c.country
FROM Competitor_Rankings cr
JOIN Competitors c ON cr.competitor_id = c.competitor_id
WHERE cr.movement = 0
ORDER BY cr.rank
"""

execute_and_display(query3, "3. Competitors with no rank movement (stable rank)")


3. Competitors with no rank movement (stable rank)
--------------------------------------------------
+----------------------------------+--------+------------+----------+----------------------+
| name                             |   rank |   movement |   points | country              |
|----------------------------------+--------+------------+----------+----------------------|
| Pavic, Mate                      |      1 |          0 |     9350 | Croatia              |
| Arevalo-Gonzalez, Marcelo        |      1 |          0 |     9350 | El Salvador          |
| Siniakova, Katerina              |      1 |          0 |    10665 | Czechia              |
| Townsend, Taylor                 |      2 |          0 |     8825 | USA                  |
| Heliovaara, Harri                |      3 |          0 |     7590 | Finland              |
| Routliffe, Erin                  |      3 |          0 |     8125 | New Zealand          |
| Ostapenko, Jelena                |      4 |          0 |  

In [101]:
query4 = """
SELECT c.country, SUM(cr.points) as total_points
FROM Competitor_Rankings cr
JOIN Competitors c ON cr.competitor_id = c.competitor_id
WHERE c.country = 'New Zealand'
GROUP BY c.country
"""

execute_and_display(query4, "4. Total points of competitors from New Zealand")


4. Total points of competitors from New Zealand
--------------------------------------------------
+-------------+----------------+
| country     |   total_points |
|-------------+----------------|
| New Zealand |          14290 |
+-------------+----------------+


In [102]:
query5 = """
SELECT c.country, COUNT(*) as competitor_count
FROM Competitors c
GROUP BY c.country
ORDER BY competitor_count DESC
"""

execute_and_display(query5, "5. Number of competitors per country")


5. Number of competitors per country
--------------------------------------------------
+--------------------------+--------------------+
| country                  |   competitor_count |
|--------------------------+--------------------|
| USA                      |                103 |
| Neutral                  |                 61 |
| Japan                    |                 56 |
| France                   |                 56 |
| Great Britain            |                 51 |
| Australia                |                 47 |
| Italy                    |                 42 |
| Czechia                  |                 39 |
| Germany                  |                 32 |
| Argentina                |                 31 |
| Netherlands              |                 31 |
| China                    |                 30 |
| India                    |                 29 |
| Spain                    |                 27 |
| Brazil                   |                 23 |
| Switzerla

In [103]:
query6 = """
SELECT c.name, cr.points, cr.rank, c.country
FROM Competitor_Rankings cr
JOIN Competitors c ON cr.competitor_id = c.competitor_id
WHERE cr.points = (SELECT MAX(points) FROM Competitor_Rankings)
ORDER BY cr.rank
"""

execute_and_display(query6, "6. Competitors with the highest points in current week")


6. Competitors with the highest points in current week
--------------------------------------------------
+---------------------+----------+--------+-----------+
| name                |   points |   rank | country   |
|---------------------+----------+--------+-----------|
| Siniakova, Katerina |    10665 |      1 | Czechia   |
+---------------------+----------+--------+-----------+
